# Clinical Preprocessing

<p>This notebook imports and organizes clinical data corresponding to the patient samples that have been evaluated for HIV-associated neurocogntive disorders and assigned a Global Deficit Score (GDS). This data is stored in a csv file that is a data dump from REDCAP, as well as a file containing the results of neurocognitive assessments. All of the patients in this study belong to the DrexelMed CARES Cohort.</p>
<p><b>Input:</b></p>
<ul>
<li>REDCAP data csv file
<li>Drexel CARES neurocognitive impairment file
</ul>
<p><b>Output:</b></p>
<ul>
<li>csv file containing all clinical information
</ul>

# Import Requirements

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import RedCap data

In [ ]:
redcap_path = '../Cohort/DrexelMedCARESCohort_DATA_LABELS_2015-10-06_1434.csv'

# Import GDS Data

In [ ]:
gds_path = '../Cohort/Drexel HIV NP Data select standardized 072815 - fixed.xlsx'
GDS_df = pd.read_excel(gds_path, headline=1)
GDS_df['PatientID'] = GDS_df['PatientID'].astype(str)
print 'GDS:', GDS_df.shape
GDS_df.head()

# Data clean-up

In [ ]:
cols = {'PatientID':'Patient',
        'Patient visit number':'Visit',
        'Date of visit':'DateOfVisit',
        'Initial CD4 count (cells/uL)':'iCD4',
        'Nadir CD4 count (cells/uL)':'nCD4',
        'Latest CD4 count (cells/uL)':'CD4',
        'Initial CD8 count (cells/uL)': 'iCD8',
        'Nadir CD8 count (cells/uL)': 'nCD8',
        'Latest CD8 count (cells/uL)': 'CD8',
        'Initial viral load (copies/mL)': 'iVL',
        'Peak viral load (copies/mL)': 'pVL',
        'Latest viral load': 'VL',
        'Total Modified Hopkins Dementia Score': 'TMHDS',
        'Gender': 'Gender',
        'Age': 'Age',
        'Years seropositive': 'Years_seropositive',
        'Race (choice=Asian)': 'Race_Asian',
        'Race (choice=American Indian/Alaska Native)': 'Race_Native_American',
        'Race (choice=Black or African American)': 'Race_Black',
        'Race (choice=Native Hawaiian or other Pacific Islander)': 'Race_Native_Hawaiian',
        'Race (choice=White)': 'Race_White',
        'Race (choice=More than one race)': 'Race_Multiple',
        'Race (choice=Unknown)': 'Race_Unknown',
        'Exposure Category (choice=Men who have sex with men (MSM))':'Exposure_MSM',
        'Exposure Category (choice=Injection drug use (IDU))':'Exposure_IDU',
        'Exposure Category (choice=Blood transfusion)':'Exposure_blood_transfusion',
        'Exposure Category (choice=Heterosexual)':'Exposure_heterosexual',
        'Exposure Category (choice=Hemophilia)':'Exposure_hemophilia', 
        'Exposure Category (choice=Perinatal)':'Exposure_perinatal',
        'Exposure Category (choice=MSM and IDU)':'Exposure_MSM_and_IDU',
        'Exposure Category (choice=Heterosexual and IDU)':'Exposure_heterosexual_and_IDU',
        'Exposure Category (choice=Other)':'Exposure_other',
        'Exposure Category (choice=ND)':'Exposure_unknown',
        'Current ART status': 'ART'}

col_order = ['Patient', 'Visit', 'DateOfVisit', 'Age', 'Gender', 'ART',
             'VL', 'iVL', 'pVL', 'CD4', 'iCD4', 'nCD4', 'CD8', 'iCD8', 'nCD8',
             'Race_Asian', 'Race_Native_American', 'Race_Black', 'Race_Native_Hawaiian',
             'Race_White', 'Race_Multiple', 'Race_Unknown', 'Exposure_MSM',
             'Exposure_IDU', 'Exposure_blood_transfusion', 'Exposure_heterosexual',
             'Exposure_hemophilia', 'Exposure_perinatal', 'Exposure_MSM_and_IDU',
             'Exposure_heterosexual_and_IDU', 'Exposure_other', 'Exposure_unknown',
             'Years_seropositive','TMHDS']

redcap_df = import_redcap_data(redcap_path)
redcap_df['PatientID'] = redcap_df['PatientID'].astype(str)
print 'REDCAP:', redcap_df.shape

clin_df = select_clinical_parameters(redcap_df, cols)
clin_df = clin_df[col_order]
print 'CLIN:', clin_df.shape
clin_df.head(5)

In [ ]:
visits = []
for idx, row in GDS_df.iterrows():
    NEURO_PATIENT = row['PatientID']
    NEURO_DATE = row['VisitDate']    
    
    clin_info = clin_df[clin_df.Patient == NEURO_PATIENT][['Patient', 'Visit', 'DateOfVisit']]
    clin_info2 = clin_info[clin_info.DateOfVisit == NEURO_DATE][['Patient', 'Visit', 'DateOfVisit']]
    if clin_info2.empty:
        #print NEURO_PATIENT, NEURO_DATE
        for a, b in clin_info.dropna().iterrows():
            days_difference = b.DateOfVisit - NEURO_DATE
            if abs(days_difference.days) < 7:
                visits.append(b.Visit)
                #print ''
    else:
        #print NEURO_PATIENT, NEURO_DATE
        visits.append(list(clin_info2['Visit'])[0])
        #print ''
        
GDS_df['Visit'] = visits
print GDS_df.shape
GDS_df[['PatientID', 'Visit', 'VisitDate', 'GDS']].head()

# Merge Data

In [ ]:
fullclinical_df = pd.merge(clin_df,GDS_df,
                  left_on = ['Patient','Visit'],
                  right_on = ['PatientID','Visit'],
                  how = 'outer')

fullclinical_df.drop(['PatientID'], axis=1, inplace=True)
print fullclinical_df.shape
fullclinical_df.head()

# Fix Errors

In [ ]:
incorrect_columns = ['iCD4','nCD4','CD4','iCD8','nCD8','CD8','iVL','pVL','VL']

for colname in incorrect_columns:
    #print colname
    corrected_values = []
    for item in list(fullclinical_df[colname]):
        l = []
        if isinstance(item, float):
            corrected_values.append(item)
        elif isinstance(item, str):
            item = item.replace('<','')
            item = item.replace('>','')
            item = item.replace(',','')
            item = item.replace('?','')
            for t in item.split():
                try:
                    l.append(float(t))
                except ValueError:
                    #pass
                    l.append(np.nan)
                except IndexError:
                    l.append(np.nan)
            #print l, item
            corrected_values.append(l[0])
        else:
            print item, type(item)
    #print len(corrected_values)
    fullclinical_df[colname] = corrected_values

print 'Full Clinical:', fullclinical_df.shape
fullclinical_df.head()

# Save clinical information in file

In [ ]:
fullclinical_df.to_csv('../Cohort/full_clinical.csv', index=False)